In [1]:
# Import machine learning library
using Flux
using Flux.Tracker
using Flux: @epochs
using Base.Test

In [2]:
# Create our training data
using ExcelFiles, CSVFiles, DataFrames

# Load normalized FIFA world rankings for each team
function loadTeamRankings(filename="../data/team-rankings.csv")
    team_rankings = load(filename) |> DataFrame;
    rankings = Dict()
    max_points = maximum(team_rankings[:Points])
    for i in 1:length(team_rankings[:Team])
        row = team_rankings[i, [:Team, :Points, :Bias]]
        rankings[row[:Team][1]] = row[:Points][1] / max_points
    end
    rankings
end

# Assign a numeric identifier (0, 1) to each team
function loadTeamIdentifiers(filename="../data/team-rankings.csv")
    names = load(filename) |> DataFrame
    const n = nrow(names)
    identifiers = Dict()
    for i in 1:n
        team = names[i, [:Team]][1][1]
        identifiers[team] = i / n
    end
    identifiers
end

# Load game results for each team
# (TeamNameA, TeamNameB, ScoreA, ScoreB)
function loadGames(filename="../data/game-results.xlsx")
    group_phase = load(filename, "Games") |> DataFrame;
    group_phase
end

# Convert teams into their numeric representation and associate with outcomes of each match
function formatDataset(identifiers, rankings, games)
    # return (x, y) tuples of input teams and output scores
    data = []
    for g in eachrow(games)
        a = g[:TeamA]
        b = g[:TeamB]
        score = g[:ScoreA]
        x1 = [identifiers[a], identifiers[b], rankings[a], rankings[b]];
        y1 = g[:ScoreA] / 10
        x2 = [identifiers[b], identifiers[a], rankings[b], rankings[a]];
        y2 = g[:ScoreB] / 10
        push!(data, (x1, y1))
        push!(data, (x2, y2))
    end
    data
end

# Scores a match between named teams 'a' and 'b'
# Returns a tuple of their respective scores
function scoreGame(model, identifiers, rankings, a, b)
    scorea = model([identifiers[a],
                    identifiers[b],
                    rankings[a],
                    rankings[b]])
    scoreb = model([identifiers[b],
                    identifiers[a],
                    rankings[b],
                    rankings[a]])
    return (scorea * 10, scoreb * 10)
end
;

In [3]:
# Load data from excel and csv files
identifiers = loadTeamIdentifiers()
rankings = loadTeamRankings()
games = loadGames()
;

In [4]:
# Convert loaded data into a more ML-friendly format
fields = [:TeamA, :TeamB, :ScoreA, :ScoreB]

# use group round games as training data
training_set = formatDataset(identifiers, rankings, games[1:48, fields])
# use round of 16 as test data
test_set = formatDataset(identifiers, rankings, games[49:56, fields])
;

In [5]:
@testset "Formatted Data" begin
    @test first(training_set)[1][1] == identifiers["Russia"]
    @test first(training_set)[1][2] == identifiers["Saudi Arabia"]
    @test first(test_set)[1][1] == identifiers["Uruguay"]
    @test first(test_set)[1][2] == identifiers["Portugal"]
end
;

Test Summary:  | Pass  Total
Formatted Data |    4      4


In [13]:
model = Chain(
    Dense(4, 4, relu),
    Dense(4, 4, relu),
    Dense(4, 1, relu)
)
# Squared loss since target values are in a range
loss(x, y) = sum((model(x) .- y).^2)
optimizer = Flux.Optimise.ADAM(params(model));

test_x, test_y = first(test_set)
observer() = @show(loss(test_x, test_y))
cb = Flux.throttle(observer, 0.5, leading=false, trailing=true)

@epochs 2000 Flux.train!(loss, training_set, optimizer, cb = cb)
;

INFO: Epoch 1
INFO: Epoch 2
INFO: Epoch 3
INFO: Epoch 4
INFO: Epoch 5
INFO: Epoch 6
INFO: Epoch 7
INFO: Epoch 8
INFO: Epoch 9
INFO: Epoch 10
INFO: Epoch 11
INFO: Epoch 12
INFO: Epoch 13
INFO: Epoch 14
INFO: Epoch 15
INFO: Epoch 16
INFO: Epoch 17
INFO: Epoch 18
INFO: Epoch 19
INFO: Epoch 20
INFO: Epoch 21
INFO: Epoch 22
INFO: Epoch 23
INFO: Epoch 24
INFO: Epoch 25
INFO: Epoch 26
INFO: Epoch 27
INFO: Epoch 28
INFO: Epoch 29
INFO: Epoch 30
INFO: Epoch 31
INFO: Epoch 32
INFO: Epoch 33
INFO: Epoch 34
INFO: Epoch 35
INFO: Epoch 36
INFO: Epoch 37
INFO: Epoch 38
INFO: Epoch 39
INFO: Epoch 40
INFO: Epoch 41
INFO: Epoch 42
INFO: Epoch 43


loss(test_x, test_y) = 0.0054320824732771306 (tracked)


INFO: Epoch 44
INFO: Epoch 45
INFO: Epoch 46
INFO: Epoch 47
INFO: Epoch 48
INFO: Epoch 49
INFO: Epoch 50
INFO: Epoch 51
INFO: Epoch 52
INFO: Epoch 53
INFO: Epoch 54
INFO: Epoch 55
INFO: Epoch 56
INFO: Epoch 57
INFO: Epoch 58
INFO: Epoch 59
INFO: Epoch 60
INFO: Epoch 61
INFO: Epoch 62
INFO: Epoch 63
INFO: Epoch 64
INFO: Epoch 65
INFO: Epoch 66
INFO: Epoch 67
INFO: Epoch 68
INFO: Epoch 69
INFO: Epoch 70
INFO: Epoch 71
INFO: Epoch 72
INFO: Epoch 73
INFO: Epoch 74
INFO: Epoch 75
INFO: Epoch 76
INFO: Epoch 77
INFO: Epoch 78
INFO: Epoch 79
INFO: Epoch 80
INFO: Epoch 81
INFO: Epoch 82
INFO: Epoch 83
INFO: Epoch 84
INFO: Epoch 85
INFO: Epoch 86
INFO: Epoch 87
INFO: Epoch 88
INFO: Epoch 89


loss(test_x, test_y) = 0.005277368113555236 (tracked)


INFO: Epoch 90
INFO: Epoch 91
INFO: Epoch 92
INFO: Epoch 93
INFO: Epoch 94
INFO: Epoch 95
INFO: Epoch 96
INFO: Epoch 97
INFO: Epoch 98
INFO: Epoch 99
INFO: Epoch 100
INFO: Epoch 101
INFO: Epoch 102
INFO: Epoch 103
INFO: Epoch 104
INFO: Epoch 105
INFO: Epoch 106
INFO: Epoch 107
INFO: Epoch 108
INFO: Epoch 109
INFO: Epoch 110
INFO: Epoch 111
INFO: Epoch 112
INFO: Epoch 113
INFO: Epoch 114
INFO: Epoch 115
INFO: Epoch 116
INFO: Epoch 117
INFO: Epoch 118
INFO: Epoch 119
INFO: Epoch 120
INFO: Epoch 121
INFO: Epoch 122
INFO: Epoch 123
INFO: Epoch 124
INFO: Epoch 125


...

loss(test_x, test_y) = 0.006477669270251137 (tracked)


INFO: Epoch 1956
INFO: Epoch 1957
INFO: Epoch 1958
INFO: Epoch 1959
INFO: Epoch 1960
INFO: Epoch 1961
INFO: Epoch 1962
INFO: Epoch 1963
INFO: Epoch 1964
INFO: Epoch 1965
INFO: Epoch 1966
INFO: Epoch 1967
INFO: Epoch 1968
INFO: Epoch 1969
INFO: Epoch 1970
INFO: Epoch 1971
INFO: Epoch 1972
INFO: Epoch 1973
INFO: Epoch 1974
INFO: Epoch 1975
INFO: Epoch 1976
INFO: Epoch 1977
INFO: Epoch 1978
INFO: Epoch 1979
INFO: Epoch 1980
INFO: Epoch 1981
INFO: Epoch 1982
INFO: Epoch 1983
INFO: Epoch 1984
INFO: Epoch 1985
INFO: Epoch 1986
INFO: Epoch 1987
INFO: Epoch 1988
INFO: Epoch 1989
INFO: Epoch 1990
INFO: Epoch 1991
INFO: Epoch 1992
INFO: Epoch 1993
INFO: Epoch 1994


loss(test_x, test_y) = 0.00570503496821925 (tracked)


INFO: Epoch 1995
INFO: Epoch 1996
INFO: Epoch 1997
INFO: Epoch 1998
INFO: Epoch 1999
INFO: Epoch 2000


loss(test_x, test_y) = 0.00580377870921123 (tracked)


In [14]:
scoreGame(model, identifiers, rankings, "Senegal", "France")

(param([1.05379]), param([1.51411]))

In [15]:
matches = [ ("Belgium", "England"),
            ("Croatia", "France"),]

for m in matches
    a = m[1]
    b = m[2]
    scorea, scoreb = scoreGame(model, identifiers, rankings, a, b)
    println("$a: $(scorea[1]), $b: $(scoreb[1])")
end

Belgium: 1.5845041464788199 (tracked), England: 1.270893947273851 (tracked)
Croatia: 1.1710320204467581 (tracked), France: 1.4939079713767183 (tracked)


In [16]:
scoreGame(model, identifiers, rankings, "France", "Saudi Arabia")

(param([2.13503]), param([0.651839]))